In [1]:
#-------------------------------------------Aitzaz Ahmad--------------------------------------------
#-------------------------------------------18I-0589-----------------------------------------------

import csv
from collections import defaultdict
import numpy as np
from numpy import random
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline

def convert_to_numeric(car_data):
    rows = len(car_data)
    cols = len(car_data[0])
    for i in range(rows):
        for j in range(cols):
            if car_data[i][j] == "vhigh":
                car_data[i][j] = 4
            elif car_data[i][j] == "high":
                car_data[i][j] = 3
            elif car_data[i][j] == "med":
                car_data[i][j] = 2
            elif car_data[i][j] == "low":
                car_data[i][j] = 1
            elif car_data[i][j] == "5more":
                car_data[i][j] = 5
            elif car_data[i][j] == "more":
                car_data[i][j] = 6
            elif car_data[i][j] == "small":
                car_data[i][j] = 1
            elif car_data[i][j] == "big":
                car_data[i][j] = 2
            elif car_data[i][j] == "unacc" :
                car_data[i][j] = 1
            elif car_data[i][j] == "acc":
                car_data[i][j] = 2
            elif car_data[i][j] == "good":
                car_data[i][j] = 3
            elif car_data[i][j] == "vgood":
                car_data[i][j] = 4
            elif car_data[i][j] == "2" or car_data[i][j] == "3" or car_data[i][j] == "4" or car_data[i][j] == "5":
                car_data[i][j] = int (car_data[i][j])
            else:
                print("car_data[", i, "][", j, "] = ", car_data[i][j])
    return car_data

def classify_data(X, split):
    trainingSet = []
    validationSet = []
    testingSet = []
    rows = len(X)
    col = len(X[0])
    for i in range(rows):
            if random.random() < split:
                trainingSet.append(X[i])
            elif random.random() >=split+0.2 :
                testingSet.append(X[i])
            else:
                validationSet.append(X[i])
    return trainingSet, validationSet, testingSet

def precision(row, matrix):
    col = matrix[:, row]
    return matrix[row, row] / col.sum()

def recall(col, matrix):
    row = matrix[col, :]
    return matrix[col, col] / row.sum()

def getPrecision(matrix):
    i, j = matrix.shape
    sum = 0
    for row in range(i):
        sum += precision(row, matrix)
    return sum / i

def getRecall(matrix):
    i, j = matrix.shape
    sum = 0
    for column in range(j):
        sum += recall(column, matrix)
    return sum / j

def getAccuracy(answer,testingdata):
    count = 0
    for i in range(len(testingdata)):
        if answer[i] == testingdata[i][-1]:
            count+=1
    return ((count/len(testingdata))*100)

def getF1ScoreMicro(matrix):
    numerator = (getRecall(matrix)*getPrecision(matrix))
    denumerator = (getPrecision(matrix)+getRecall(matrix)) 
    return (2*numerator/denumerator)

def getF1ScoreMacro(matrix):
    recallsForEachClass = []
    precisionsForEachClass = []
    i, j = matrix.shape
    sum = 0
    
    for column in range(j):
        recallsForEachClass.append(recall(column, matrix))
        precisionsForEachClass.append(precision(column,matrix))
    for i in range(len(recallsForEachClass)):
        numerator = (recallsForEachClass[i]*precisionsForEachClass[i])
        denominator = (precisionsForEachClass[i]+recallsForEachClass[i])
        sum_temp = 2*numerator/denominator
        sum += sum_temp
        print("F1-Score for class ",i," :", sum_temp)
    return sum/len(recallsForEachClass)


def euclideanDistance(sample_1, sample_2):
    distance = 0
    for i in range(len(sample_1)-1):
        distance += pow((sample_1[i] - sample_2[i]), 2)
    return math.sqrt(distance)


def get_neighbors(row,trainingdata,k):
    distances = []
    neighbors = []
    for i in range(len(trainingdata)):
        distances.append((trainingdata[i],euclideanDistance(row,trainingdata[i])))
    distances.sort(key=lambda x:x[1])
    for i in range(k):
        neighbors.append(distances[i][0])
        
    return neighbors

def findMostOccuredClass(neighbors):
    a = []
    for i in range(len(neighbors)):
        a.append(neighbors[i][-1])
    return max(set(a), key=a.count)

def getAccuracy(answer,testingdata):
    count = 0
    for i in range(len(testingdata)):
        if answer[i] == testingdata[i][-1]:
            count+=1
    return ((count/len(testingdata))*100)


def get_value_of_k(training_data, validation_data):
    prevAccuracy = 0
    currAccuracy = 0
    curr_k = 2
    for k in range(2,8):
        answers=[]
        for i in range(len(validation_data)):
            neighbors = get_neighbors(validation_data[i], training_data, k)
            answers.append(findMostOccuredClass(neighbors))
        accuracy = getAccuracy(answers,validation_data)
#        print ('Accuracy for Validation Set on k = ',k," is:", accuracy)
        if accuracy > currAccuracy:
#            print("This was greater than prev accuracy of ", prevAccuracy, " so now curracuuracy has changed and temp_k=",k)
            currAccuracy = accuracy
            temp_k = k
        if(abs(prevAccuracy - accuracy)<0.1):
            return temp_k
        prevAccuracy = accuracy
    return 3

def nested_dict(n, type):
    if n == 1:
        return defaultdict(type)
    else:
        return defaultdict(lambda: nested_dict(n-1, type))


def getConfusionMatrix(answer,testingdata,size,matrix):
    dic = nested_dict(2, int)
    for i in matrix:
        for j in matrix:
            dic[i][j] = 0
    for i in matrix:
        for k in range(len(testingdata)):
                for j in matrix:
                    if(testingdata[k][-1] == i and answer[k] == j):
                        dic[i][j]+=1
    
    array = np.arange(size*size).reshape(size,size)
    m = 0
    for i in matrix:
        n = 0
        for j in matrix:
            array[m][n] = dic[i][j]
            n+=1
        m+=1
    return array


def main():
    with open("car.data", 'r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter = ",")
        dataSet = list(csv_reader)  
    dataSet = convert_to_numeric(dataSet)
    training_data, validation_data, testing_data = classify_data(dataSet, 0.5)
    K = get_value_of_k(training_data, validation_data)
    print("Got value of k which is ", K)
    
    answers=[]        
    for i in range(len(testing_data)):
        neighbors = get_neighbors(testing_data[i], training_data, K)
        answers.append(findMostOccuredClass(neighbors))
    print("Got answers")
    
    x = np.array(answers)
    matrix = np.unique(x)
    confusion_matrix = getConfusionMatrix(answers,testing_data,len(matrix),matrix)
    print("Confusion Matrix")
    print(confusion_matrix)
    
    accuracy = getAccuracy(answers,testing_data)
    print ('Accuracy:', accuracy)
    print('Recall:',getRecall(confusion_matrix))
    print("precision:", getPrecision(confusion_matrix))
    print("F1Score Micro:", getF1ScoreMicro(confusion_matrix))
    print("F1Score Macro:", getF1ScoreMacro(confusion_matrix))
    
if __name__ == "__main__":
    main()

Got value of k which is  3
Got answers
Confusion Matrix
[[180   8   1   0]
 [ 11  51   0   0]
 [  1   7   1   1]
 [  0   6   1   3]]
Accuracy: 86.71586715867159
Recall: 0.5437403993855606
precision: 0.6822916666666667
F1Score Micro: 0.6051873415111306
F1-Score for class  0  : 0.9448818897637795
F1-Score for class  1  : 0.7611940298507464
F1-Score for class  2  : 0.15384615384615383
F1-Score for class  3  : 0.4285714285714285
F1Score Macro: 0.572123375508027
